In [1]:
#!pip install matplotlib-venn
import numpy as np
import pandas as pd
import zipfile

Cargamos los datos 

In [2]:
year = 2015
category = 'Programming'
name = "Stack Overflow Developer Survey Responses.csv"

zf = zipfile.ZipFile(f'../surveys/stack-overflow-developer-survey-{year}.zip') 

devs = pd.read_csv(zf.open(f'{year} {name}'), low_memory=False, header=1)

In [3]:
current_columns = [col for col in devs.columns if col.startswith("Current Lang & Tech:")]
future_columns = [col for col in devs.columns if col.startswith("Future Lang & Tech:")]

In [4]:
lang_worked = devs[current_columns].notnull().astype('int')
lang_worked.columns = lang_worked.columns.str.replace(r"Current Lang & Tech: ", "")
lang_want = devs[future_columns].notnull().astype('int')
lang_want.columns = lang_worked.columns.str.replace(r"Future Lang & Tech: ", "")

In [5]:
devs['Compensation: midpoint'] = pd.to_numeric(devs['Compensation: midpoint'], errors='coerce')

In [6]:
comps = lang_worked.multiply( devs['Compensation: midpoint']/1000, axis="index").round(decimals=0).replace(0, np.NaN).median().astype('int')

In [7]:
worked = lang_worked.sum() / len(lang_worked)
wanted = lang_want.sum() / len(lang_want)

In [8]:
w_df = pd.DataFrame(worked, columns= ['Worked'])
w_df['Worked_per'] = (w_df['Worked'] * 100).round(1)
wa_df = pd.DataFrame(wanted, columns= ['Wanted'])
wa_df['Wanted_per'] = (w_df['Worked'] * 100).round(1)
langs = pd.concat([w_df, wa_df, comps], axis=1)
langs = pd.concat([w_df, wa_df, comps], axis=1)
langs['Year'] = year
langs.rename(columns = {0:'Salary'}, inplace = True)
langs['Category'] = category

In [9]:
langs.to_csv(f'../out/{year}.csv', index=True, index_label='Language')